# Step 2: Finding Relevant Data

Add your API key to the cell below then run it.

In [1]:
import openai
openai.api_key = "sk-proj-jCMuei3DUXDlGchRhc0YT3BlbkFJlNgyekcYUyH2OWhID6zA"

The code below loads in the embeddings you previously created. Run it as-is.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("embeddings.csv", index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df

,text,embeddings
0,"On 6 February 2023, at 04:17 TRT (01:17 UTC), ...","[-0.007893280126154423, -0.014928885735571384,..."
1,The Mw 7.8 earthquake is the largest in Turkey...,"[0.00019082157814409584, -0.022295519709587097..."
2,There was widespread damage in an area of abou...,"[-0.0001506179105490446, -0.01705605536699295,..."
3,"The confirmed death toll in Turkey was 53,537;...","[0.0013410564279183745, -0.02500694990158081, ..."
4,"Damaged roads, winter storms, and disruption t...","[-0.018138449639081955, -0.014602766372263432,..."
...,...,...
194,The International Seismological Centre has a b...,"[-0.004581885877996683, -0.009613040834665298,..."
195,The International Seismological Centre has a b...,"[-0.00487518822774291, -0.0168690737336874, 0...."
196,"Erdik, M., Tümsa, M. B. D., Pınar, A., Altunel...","[-0.006733294576406479, -0.04049864038825035, ..."
197,"""Kahramanmaraş Supersite science page"". Group ...","[0.003666419070214033, -0.007841319777071476, ..."


## TODO 1: Create Embeddings for the User's Question

In the previous exercise, you were given the code to create embeddings for everything in the dataset. Now your task is to create embeddings for just one string: the user's question. Assign the result to the variable `question_embeddings`. This variable should contain a list of 1,536 floating point numbers, and the provided code will print the first 100 once `question_embeddings` has been created correctly.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """What were the estimated damages of the 2023 \
Turkey-Syria earthquake?"""

# Generate the embedding response
response = openai.Embedding.create(
    input=USER_QUESTION,
    engine=EMBEDDING_MODEL_NAME
)

# Extract the embeddings from the response
question_embeddings = response["data"][0]["embedding"]

print(question_embeddings[:100])
```

</details>

In [4]:
from openai.embeddings_utils import get_embedding

In [5]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
USER_QUESTION = """What were the estimated damages of the 2023 \
Turkey-Syria earthquake?"""

# Generate the embedding response
response = get_embedding(USER_QUESTION, engine=EMBEDDING_MODEL_NAME)

# Extract the embeddings from the response
question_embeddings = response

print(question_embeddings[:100])

[0.005581929348409176, -0.024909110739827156, 0.0022204634733498096, -0.012141861021518707, -0.021516041830182076, 0.0025265049189329147, -0.03382423147559166, -0.013119862414896488, 0.002315269783139229, -0.014889580197632313, 0.01624680683016777, 0.04444253444671631, -0.01022577565163374, -0.013153128325939178, 0.015062560327351093, -0.005322459619492292, 0.012421289458870888, -0.013053331524133682, 0.008283077739179134, -0.0057915011420845985, -0.007451443001627922, 0.011496512219309807, 0.012407983653247356, -0.009034874849021435, 0.015115784481167793, 0.03217426687479019, 0.008070179261267185, -0.002443341538310051, -0.0008981650462374091, -0.012108595110476017, 0.005545337218791246, -0.006573237478733063, -0.023538576439023018, 0.0151823153719306, -0.032413776963949203, -0.007544586434960365, 0.0068925851956009865, -0.0048234788700938225, 0.023378903046250343, -0.00795042421668768, 0.010125978849828243, 0.028634831309318542, 0.013000107370316982, -0.00770425982773304, -0.01749758

## TODO 2: Find Cosine Distances

Create a new list called `distances`, which represents the cosine distances between `question_embeddings` and each value in the `'embeddings'` column of `df`.

If you're getting stuck, you can click to reveal the solution then copy and paste this into the cell below.

---

<details>
    <summary style="cursor: pointer"><strong>Solution (click to show/hide)</strong></summary>

```python
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = distances_from_embeddings(
    question_embeddings,
    df["embeddings"],
    distance_metric="cosine"
)

print(distances[:100])
```

</details>

In [ ]:
from openai.embeddings_utils import distances_from_embeddings

# Create a list containing the distances from question_embeddings
distances = 

print(distances[:100])

## Sorting by Distance

The code below uses the `distances` list to update `df` then sorts `df` to find the most related rows. Shorter distance means more similarity, so we'll use an ascending sorting order. Run the cell below as-is.

In [ ]:
df["distances"] = distances
df.sort_values(by="distances", ascending=True, inplace=True)
df.head(5)

Great, now we have the dataset sorted from most relevant to least relevant! Let's save this as a CSV so we can load it in the next step and compose a custom prompt.

Run the cell below as-is.

In [ ]:
df.to_csv("distances.csv")